In [ ]:
import numpy as np
import nibabel as nib
import os as os
import scipy as scipy
from scipy.spatial import distance
from scipy.spatial.transform import Rotation as R
import pandas as pd
import csv as csv

In [ ]:
subjects = ['0083', '0085', '0089', '0099', '0101', '0106', '0110', '0115', '0136', '0139', '0141', '0152', '0161', '0164', '0168', '0171', '0176', '0200', '0238', '0282','0160','0181','0184','0192','0218','0224','0266','0268','0272','0276']

We implement a class for calculating RMSE for both the whole field of view (FOV) of the scan, as well as when masking only brain regions (RMSE and RMSE_Brain respectively). Statistics are written out in the system to a .csv file.

In [ ]:
class RMSE_Brain:
    import numpy as np
    import nibabel as nib
    import os as os
    from scipy.spatial import distance
    error = 0
    error2 = 0
    #load data, image files are in

    def __init__(self,subjID):
        self.subjID =str(subjID)
        cPET = nib.load('2c_masked_{}.nii.gz'.format(subjID))
        rawPET=nib.load('raw_masked_{}.nii.gz'.format(subjID))
        cPET_data = cPET.get_fdata()
        rawPET_data = rawPET.get_fdata()
        cPET_NAN = np.isnan(cPET_data)
        rawPET_NAN = np.isnan(rawPET_data)
        cPET_data[cPET_NAN] = 0
        rawPET_data[rawPET_NAN] = 0
        
        mask_dims = np.count_nonzero(cPET_data)
        raw_dims = np.count_nonzero(rawPET_data)
        #m=cPET_data.shape[0]*cPET_data.shape[1]*cPET_data.shape[2]
        
        m=mask_dims
        
        #compute differences
        self.raw_difference = np.sum(np.absolute(cPET_data - rawPET_data))/m
        difference = np.square(cPET_data - rawPET_data)
        error=np.sum(difference)/m
        
        self.mean_raw = np.sum(rawPET_data)/m
        self.mean_CT = np.sum(cPET_data)/m
        
        self.raw=np.sqrt(error)

    def dims(self):
        self.cPETd=cPET.dataobj
        self.tPETd=tPET.dataobj
        self.rawPETd=rawPET.dataobj
        return print(cPET_data.shape)
        return print(rawPET_data.shape)
    def normalize(self):
        cPET_intensity = np.nansum(cPET_data)
        rawPET_intensity = np.nansum(rawPET_data)
        tPET_intensity = np.nansum(tPET_data)
        t6PET_intensity = np.nansum(t6PET_data)
        rawPET_norm = rawPET_data * (cPET_intensity/rawPET_intensity)
        tPET_norm = tPET_data * (cPET_intensity/tPET_intensity)
        t6PET_norm = t6PET_data * (cPET_intensity/t6PET_intensity)
        return print('cPET_intensity: ' + str(cPET_intensity))
        return print('rawPET_intensity: ' + str(rawPET_intensity) + ' Normalized: ' + str(np.nansum(rawPET_norm)))
        return print('tPET_intensity: ' + str(tPET_intensity) + ' Normalized: ' + str(np.nansum(tPET_norm)))

    def show(self):
        return print('temp error: ' + str(error2) + '\n' + 'raw error: ' + str (error))
    def write(self):
        import csv
        line_add= [self.subjID,self.raw,self.temp,str(self.temp < self.raw),self.t6MI,self.t6CR,str((self.t6MI < self.temp) or (self.t6CR < self.temp)),str((self.t6MI < self.raw) or (self.t6CR < self.raw))]
        with open('RMSE.csv', 'a') as data:
            writer = csv.writer(data)
            writer.writerow(line_add)
        return print('Written to RMSE.csv: {}'.format(line_add))
    def percent_error(self):

        return print((self.raw_difference/self.mean_CT)*100)
    
class RMSE:
    import numpy as np
    import nibabel as nib
    import os as os
    from scipy.spatial import distance
    error = 0
    error2 = 0
    #load data, image files are in

    def __init__(self,subjID):
        self.subjID =str(subjID)
        cPET = nib.load('cPET_in_MRI_{}.nii.gz'.format(subjID))
        rawPET=nib.load('PET_motion_MERGE_reg_raw_{}.nii.gz'.format(subjID))
        tPET=nib.load('tmaPET_to_MRI_{}.nii.gz'.format(subjID))
        tPET_ssMI=nib.load('PET_to_subjspacetemp_MI_{}.nii.gz'.format(subjID))
        tPET_ssCR=nib.load('PET_to_subjspacetemp_CR_{}.nii.gz'.format(subjID))
        tPET_data = tPET.get_fdata()
        cPET_data = cPET.get_fdata()
        rawPET_data = rawPET.get_fdata()
        tPET_ssCR_data = tPET_ssCR.get_fdata()
        tPET_ssMI_data = tPET_ssMI.get_fdata()
        ssCR_NAN = np.isnan(tPET_ssCR_data)
        ssMI_NAN = np.isnan(tPET_ssMI_data)
        tPET_NAN = np.isnan(tPET_data)
        cPET_NAN = np.isnan(cPET_data)
        rawPET_NAN = np.isnan(rawPET_data)
        tPET_data[tPET_NAN] = 0
        cPET_data[cPET_NAN] = 0
        rawPET_data[rawPET_NAN] = 0
        tPET_ssCR_data[ssCR_NAN] = 0
        tPET_ssMI_data[ssMI_NAN] = 0
        
        mask_dims = np.count_nonzero(cPET_data)
        assert(mask_dims==np.count_nonzero(tPET_data))
        m=mask_dims
        
        #compute differences
        difference = np.square(cPET_data - rawPET_data)
        error=np.sum(difference)
        difference2 = np.square(cPET_data - tPET_data)
        error2=np.sum(difference2)
        difference3 = np.square(cPET_data - tPET_ssMI_data)
        error3=np.sum(difference3)
        difference4 = np.square(cPET_data - tPET_ssCR_data)
        error4=np.sum(difference4)
        
        self.raw=error/m
        self.temp=error2/m
        self.t6MI=error3/m
        self.t6CR=error4/m
    def dims(self):
        self.cPETd=cPET.dataobj
        self.tPETd=tPET.dataobj
        self.rawPETd=rawPET.dataobj
        return print(cPET_data.shape)
        return print(rawPET_data.shape)
    def normalize(self):
        cPET_intensity = np.nansum(cPET_data)
        rawPET_intensity = np.nansum(rawPET_data)
        tPET_intensity = np.nansum(tPET_data)
        t6PET_intensity = np.nansum(t6PET_data)
        rawPET_norm = rawPET_data * (cPET_intensity/rawPET_intensity)
        tPET_norm = tPET_data * (cPET_intensity/tPET_intensity)
        t6PET_norm = t6PET_data * (cPET_intensity/t6PET_intensity)
        return print('cPET_intensity: ' + str(cPET_intensity))
        return print('rawPET_intensity: ' + str(rawPET_intensity) + ' Normalized: ' + str(np.nansum(rawPET_norm)))
        return print('tPET_intensity: ' + str(tPET_intensity) + ' Normalized: ' + str(np.nansum(tPET_norm)))
    def show(self):
        return print('temp error: ' + str(error2) + '\n' + 'raw error: ' + str (error))
    def write(self):
        import csv
        line_add= [self.subjID,self.raw,self.temp,str(self.temp < self.raw),self.t6MI,self.t6CR,str((self.t6MI < self.temp) or (self.t6CR < self.temp)),str((self.t6MI < self.raw) or (self.t6CR < self.raw))]
        with open('RMSE.csv', 'a') as data:
            writer = csv.writer(data)
            writer.writerow(line_add)
        return print('Written to RMSE.csv: {}'.format(line_add))

In [ ]:
subjects = ["195","196","198","199","201","202",
            "210","211","212","213","214","216",
            "230","231","232","233","234","235",
            "257","259","264","265","120","121",
            "122","123","124","125","126","127",
            "83","85","89","99","101","106","110",
            "115","136","139","141","152","161","164",
            "168","171","176","200","238","282","160",
            "181","184","192","218","224","266","268",
            "272","276"]

for i in subjects:
    if len(i) < 4:
        if len(i) ==2:
            subjID = "00" + i
        else:
            subjID = "0" + i
    y = RMSE_Brain(subjID)
    print(subjID)
    y.percent_error()

Translational differences are calculated by simply taking the 4th column of the transformation matrix generated by flirt for each registration.

To calculate Euler angles, we use the 'Rotation' Package from scipy. Derivation of euler angles are explained in detail in:

Tilma, Todd, and E. C. G. Sudarshan. "Generalized Euler angle parametrization for SU (N)." Journal of Physics A: Mathematical and General 35.48 (2002): 10467.

In [ ]:
for i in subjects:
    CT= (pd.read_csv('2c_in_MRI_{}.csv'.format(i),header=None,usecols=[0,1,2,3])).values
    raw = (pd.read_csv('invol2refvol_raw_{}.csv'.format(i),header=None,usecols=[0,1,2,3])).values
    rerunraw = (pd.read_csv('3rd_run_raw_{}.csv'.format(i),header=None,usecols=[0,1,2,3])).values
    temp = (pd.read_csv('PET_to_temp_{}.csv'.format(i),header=None,usecols=[0,1,2,3])).values
    MI = (pd.read_csv('PET_tosubjspacetemp_MI_{}.csv'.format(i),header=None,usecols=[0,1,2,3])).values
    CR = (pd.read_csv('PET_tosubjspacetemp_CR_{}.csv'.format(i),header=None,usecols=[0,1,2,3])).values
    print(CT)
    print(raw)
    print(rerunraw)
    print(temp)
    print(MI)
    print(CR)

    trans_raw=np.sum(np.sqrt(np.square(CT-raw))[:,3])
    print(trans_raw)
    trans_rerunraw=np.sum(np.sqrt(np.square(CT-rerunraw))[:,3])
    print(trans_rerunraw)
    trans_temp=np.sum(np.sqrt(np.square(CT-temp))[:,3])
    print(trans_temp)
    trans_MI=np.sum(np.sqrt(np.square(CT-MI))[:,3])
    print(trans_MI)
    trans_CR=np.sum(np.sqrt(np.square(CT-CR))[:,3])
    print(trans_CR)
    
    trans = [trans_raw,trans_rerunraw,trans_temp,trans_MI,trans_CR]
    
    trans_dict = {
        trans_raw:'raw',trans_rerunraw:'rerun_raw',trans_temp:'temp',trans_MI:'MI',trans_CR:'CR'
    }
    
    rot_CT=CT[:3,:3]
    rot_raw=raw[:3,:3]
    rot_rerunraw=rerunraw[:3,:3]
    rot_temp=temp[:3,:3]
    rot_MI=MI[:3,:3]
    rot_CR=CR[:3,:3]
    r_CT = R.from_dcm(rot_CT)
    r_raw = R.from_dcm(rot_raw)
    r_rerunraw = R.from_dcm(rot_rerunraw)
    r_temp = R.from_dcm(rot_temp)
    r_MI = R.from_dcm(rot_MI)
    r_CR = R.from_dcm(rot_CR)

    euler_CT = r_CT.as_euler('zyx',degrees=True)
    euler_raw = r_raw.as_euler('zyx',degrees=True)
    euler_rerunraw = r_rerunraw.as_euler('zyx',degrees=True)
    euler_temp = r_temp.as_euler('zyx',degrees=True)
    euler_MI = r_MI.as_euler('zyx',degrees=True)
    euler_CR = r_CR.as_euler('zyx',degrees=True)

    eulers = [euler_CT,euler_raw,euler_rerunraw,euler_temp,euler_MI,euler_CR]
    print(str(eulers))

    euler_differences = [euler_CT-euler_raw,euler_CT-euler_rerunraw,euler_CT-euler_temp,euler_CT-euler_MI,
                         euler_CT-euler_CR]
    euler_diff_dict = {np.sum(np.sqrt(np.square(euler_CT-euler_raw))):'raw',np.sum(np.sqrt(np.square(euler_CT-euler_rerunraw))):'rerun_raw',np.sum(np.sqrt(np.square(euler_CT-euler_temp))):'temp',np.sum(np.sqrt(np.square(euler_CT-euler_MI))):'6dofMI',
                         np.sum(np.sqrt(np.square(euler_CT-euler_CR))):'6dofCR'}
    euler_differences_sum = [np.sum(np.sqrt(np.square(euler_CT-euler_raw))),np.sum(np.sqrt(np.square(euler_CT-euler_rerunraw))),
                             np.sum(np.sqrt(np.square(euler_CT-euler_temp))),np.sum(np.sqrt(np.square(euler_CT-euler_MI))),np.sum(np.sqrt(np.square(euler_CT-euler_CR)))]
    print(str(euler_differences_sum))
    line_add = [i,'Transformation Differences:',trans_raw,trans_rerunraw,trans_temp,trans_MI,trans_CR,trans_dict[min(trans)],
                'Euler Angle Differences:',euler_differences_sum[0],euler_differences_sum[1],euler_differences_sum[2],euler_differences_sum[3],
                euler_differences_sum[4],euler_diff_dict[min(euler_differences_sum)]]
    with open('Transformation.csv', 'a') as data:
        writer = csv.writer(data)
        writer.writerow(line_add)